# Palmer Amaranth (Amaranthus palmeri) Growth Stage Detection
This Google Colab notebook tutorial is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by WongKinYiu. For additional context, the original authors recommend that you follow along in this notebook while reading the blog post on [how to train YOLOv7](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/), concurrently. This can be used to train and evaluate YOLOv7 on the Palmer amaranth Growth Stage (PAGS8) dataset. The dataset is available for download from [Weed-AI](). Prior to use, the dataset should be setup using the instructions from the PAGS8 repository.

It accompanies the preprint: **Multi-growth stage plant recognition: a case study of Palmer amaranth (Amaranthus palmeri) in cotton (Gossypium hirsutum)** available on [arXiv](https://arxiv.org/abs/2307.15816). Please consider citing this preprint if you use the work in your research.

Models were trained using Google Colab Pro+ with access to an NVIDIA A100-SXM4-40GB.

This notbook consists of three parts:
1. Setup
2. Training
3. Evaluation

For additional documentation, training options and inference procedures please visit the official YOLOv7 repository.

(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)

<img src="https://github.com/geezacoleman/Palmer-detection/assets/51358498/6040227e-a072-43bf-a789-72e0833f3168"/>


# 1. Setup

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import necessary packages
import torch
import random

random.seed(0)

from datetime import datetime

DATE = datetime.now().strftime('%Y%m%d')
IMAGE_SIZE = 1280

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
rootDir = '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/data'
%cd '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/'

In [ ]:
%cd yolov7
!pip install -r requirements.txt

## Download model weights
NOTE: The pretrained models used here are from YOLOv7 release [0.0.1](https://github.com/WongKinYiu/yolov7/releases/tag/v0.1)

In [ ]:
# download checkpoints
%cd '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/yolov7'
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x_training.pt

Use weights and biases (WandB) if needed for managing and tracking training progress.

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

# 2. Training

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Palmer-detection/yolov7'
models = ['x', ''] # train X and Original models
classes = ['1', '8']

for model in models:
  for class_num in classes:
    for fold in range(0, 5):
      yaml_path = f"{class_num}cls_fold_{fold}.yaml"
      full_name = f'{DATE}_yolov7{model}_B8_F{fold}_{class_num}cls'
      
      print(full_name)
      !python train.py --workers 8 --device 0 --img {IMAGE_SIZE} --cfg cfg/training/yolov7{model}.yaml --hyp data/hyp.scratch.custom.yaml --batch 8 --epochs 30 --data data/{data_path} --weights yolov7{model}_training.pt --name {full_name}

# 3. Evaluation

In [ ]:
import os
train_path = "/content/drive/My Drive/Colab Notebooks/Palmer-detection/yolov7/runs/train"
for run_name in os.listdir(train_path):
  if 'yolo' not in run_name:
    print(f'skipping {run_name}')
  
  else:
    fold = run_name.split("_")[-3][-1]
    class_num = run_name.split("_")[-2][:-3]
    
    yaml_path = f"{class_num}cls_fold_{fold}.yaml"
  
    !python test.py --img {IMAGE_SIZE} --weights runs/train/{run_name}/weights/best.pt --data data/{yaml_path} --name {run_name} --task 'test' --batch 8  --iou-thres 0.6 --conf-thres 0.001 --verbose  --v5-metric  --save-metrics
